In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import re
import json
from nltk.corpus import stopwords
import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
stopwords = stopwords.words('english')
tqdm.pandas()
from english_words import english_words_lower_alpha_set as word_list
from textblob import TextBlob
corpus = []

In [2]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [3]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

In [4]:
mispell_dict = {"can't":"cannot","ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

In [5]:
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [6]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [7]:
# Some imports and global variables related to lemmatization
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from collections import defaultdict

tree_tag_map = defaultdict(lambda : wn.NOUN)
tree_tag_map['J'], tree_tag_map['V'], tree_tag_map['R'] = wn.ADJ, wn.VERB, wn.ADV
lemmatize_function = WordNetLemmatizer()
lemmatized_words_dp, LEMMATIZE_DP_LIMIT = {}, 500000

In [8]:
def do_lemmatize(token):
    if lemmatized_words_dp.get(token):
        return lemmatized_words_dp[token]
    new_token = [lemmatize_function.lemmatize(token, tree_tag_map[tag[0]]) for token, tag in pos_tag([token])]
    if len(lemmatized_words_dp) != LEMMATIZE_DP_LIMIT:
        lemmatized_words_dp[token] = new_token[0]
    return new_token[0]

In [9]:
import re
import string
remove = string.punctuation
remove = remove.replace(".", "") # don't remove fullstops
remove = remove.replace("-", "") # don't remove hyphens
pattern = r"[{}]".format(remove) # create the pattern

In [10]:
def clean_text(x):
    x = str(x).lower()
    x = replace_typical_misspell(x)
    x = re.sub("@[A-Za-z0-9]+"," ", x) # remove user IDs
    x = re.sub("(http://.*?\s)|(http://.*)",' ',str(x)) #remove http links in the text
    x = re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",' ',str(x)) # remove IP addresses
    #for punct in puncts:
        #if punct in x:
            #x = x.replace(punct, f' {punct} ')
    x = clean_numbers(x)
    x = remove_emoji(x)
    x = re.sub(pattern, "", x) 
    x = x.split(" ")
    x = [do_lemmatize(w) for w in x]
    x = " ".join(x)
    x =" ".join(x.split())
    return x

In [11]:
train = pd.read_csv("unclean/train.csv")#.head(500)
test = pd.read_csv("unclean/test.csv")#.head(500)
val = pd.read_csv("unclean/val.csv")#.head(500)

In [12]:
train["heading"] = train["heading"].progress_apply(lambda x: clean_text(x))
train["body"] = train["body"].progress_apply(lambda x: clean_text(x))
train["heading"].progress_apply(lambda x: corpus.extend(x.split(".")))
train["body"].progress_apply(lambda x: corpus.extend(x.split(".")))
test["heading"] = test["heading"].progress_apply(lambda x: clean_text(x))
test["body"] = test["body"].progress_apply(lambda x: clean_text(x))
val["heading"] = val["heading"].progress_apply(lambda x: clean_text(x))
val["body"] = val["body"].progress_apply(lambda x: clean_text(x))

  0%|          | 0/93652 [00:00<?, ?it/s]

  0%|          | 0/93652 [00:00<?, ?it/s]

  0%|          | 0/93652 [00:00<?, ?it/s]

  0%|          | 0/93652 [00:00<?, ?it/s]

  0%|          | 0/11707 [00:00<?, ?it/s]

  0%|          | 0/11707 [00:00<?, ?it/s]

  0%|          | 0/11706 [00:00<?, ?it/s]

  0%|          | 0/11706 [00:00<?, ?it/s]

In [13]:
train.to_csv("clean/train.csv",index=False)
test.to_csv("clean/test.csv",index=False)
val.to_csv("clean/val.csv",index=False)

In [14]:
print("Done")

Done


In [15]:
corpus

['two gop congressman say suspect terrorist caught cross u',
 's',
 '–mexico border',
 'jasmine tridevil woman with three breast denies surgery hoax claim',
 'drake late project be a playlist — a format that hadn’t be use to express an artistic vision until now',
 'police woman “trained” to poison kim jong un’s half brother',
 'u official russian drone may have turn camera off right before syrian attack',
 'watchdog there be incontrovertible evidence sarin use in syria chemical weapon attack',
 'apple to shut down beat music not so apple say',
 'a reminder that christina aguilera have a christmas album and it be amaze',
 'florida woman get third breast surgically implant',
 'isil video claim beheading of kidnap journalist james foley',
 'islamic state claim to have behead american journalist',
 'saudi arabian airline to separate gender after men complain about bloke sit next to their wife',
 'nun give birth intend to keep baby',
 'man girlfriend pours acid on his penis after he covertl

In [16]:
df = pd.read_csv("clean/train.csv")

In [17]:
import json
with open("./clean/corpus.json","w") as f:
    json.dump(corpus,f,indent=4)
    

In [18]:
df

,heading,body,label
0,two gop congressman say suspect terrorist caug...,a -year-old canadian-israeli woman who travel ...,1
1,jasmine tridevil woman with three breast denie...,saudi arabia’s national airline carrier be pla...,1
2,drake late project be a playlist — a format th...,for almost a decade drake have be a star and a...,0
3,police woman “trained” to poison kim jong un’s...,kuala lumpur malaysia -- the two woman suspect...,0
4,u official russian drone may have turn camera ...,david choi business insider 8.. 1 facebook lin...,0
...,...,...,...
93647,there’s a catch with opec’s plan to cut produc...,’ ’ ’ opec ’ ’ ’ ’ additionally also agree to ...,0
93648,no pumpkin spice condom be not a real thing,the man know a jihadi john. photo reuters the ...,1
93649,video show isil beheading of photojournalist j...,a touch tribute to the victim of the charlie h...,1
93650,canada probe michael zehaf-bibeau a possible s...,the man behind the ottawa shooting be a canadi...,0
